In [1]:
# Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import random
import community as comm
from gensim.models import Word2Vec

from sklearn.cluster import KMeans
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

import warnings
warnings.filterwarnings('ignore')

import torch 
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
import torchvision
import torchvision.transforms as transforms


In [2]:
# Function to create a GNN module to run a 2-layer network for GCN, GraphSage and GAT

class Net(torch.nn.Module):
    def __init__(self, in_feats, val, dropout_rate, classes):
        super(Net, self).__init__()
        
        if val == 0:
            self.conv1 = GCNConv(in_feats, 16)
            self.conv2 = GCNConv(16, classes)
        elif val == 1:
            self.conv1 = SAGEConv(in_feats, 16)
            self.conv2 = SAGEConv(16, classes)
        else:
            self.conv1 = GATConv(in_feats, 16)
            self.conv2 = GATConv(16, classes)
            
        self.dropout_rate = dropout_rate
 

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [3]:
# Function to get the node embeddings by running desired GNN model

def gnn_embed(dataset, gnn_val=0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(in_feats = dataset.num_node_features, val = gnn_val, dropout_rate=0.05, classes = dataset.num_classes).to(device)
    data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        _, pred = model(data).max(dim=1)
        correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
        acc = correct / data.train_mask.sum().item()
        print('Epoch: %d, Train Accuracy: %.4f'%(epoch,acc))

        if acc == 1:
            break

    # Freeze the last layer and get node embeddings
    model.conv2.eval()
    with torch.no_grad():
        node_embeddings = model.conv1(data.x, data.edge_index)
        node_embeddings = F.relu(node_embeddings)
        node_embeddings = model.conv2(node_embeddings, data.edge_index)
        node_embeddings = node_embeddings.cpu().numpy()

    # Convert node embeddings to dictionary
    node_dict = {}
    for i in range(len(node_embeddings)):
        node_dict[i] = node_embeddings[i]

    # Get the Test Accuracy
    _, pred = model(data).max(dim=1)
    correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    print('\nTest Accuracy: {:.4f}'.format(acc))
        
    # Entire Dataset accuracy
    _, pred = model(data).max(dim=1)
    correct = float (pred.eq(data.y).sum().item())
    acc = correct / len(data.x)
    print('\nTotal Accuracy: {:.4f}\n'.format(acc))
        
    # Return node embedding    
    return node_dict, acc


In [4]:
# Function to get the Predicted probabilities of the edges based on Link Prediction heuristic

def makeLinkPrediction(G, embeddings):
    # converting embedding to a numpy array
#     X = [[0] for i in range(graph.number_of_nodes())]
#     for i in range(0, graph.number_of_nodes()):
#         X[i] = embeddings[str(i+1)]
#     X = np.array(X)
    X = np.array([embeddings[i] for i in embeddings.keys()])
    
    Xd = []
    Yd = []
    Xdd = []
    Ydd = []
    count = 0
    # for all vertices
    vlist = []
#     for u in range(1, G.number_of_nodes() + 1):
    for u in range(G.number_of_nodes()):
#         Nu = [int(nn) for nn in list(G.neighbors(str(u)))]
        Nu = [nn for nn in list(G.neighbors(u))]
        count += len(Nu)
        cn = 0
        totalns = 0
        # for all neighbors of u
        for n in Nu:
            x = []
            if n > u and ((u,n) not in vlist and (n,u) not in vlist):
                for d in range(len(X[0])):
                    x.append(X[u][d] - X[n][d]) # distance between the embeddings of u and its neighbor n
                Xd.append(x)
                Yd.append(1) # positive sample (edge present)
                totalns += 1
                vlist.append((u,n) if (u,n) in list(G.edges()) else (n,u))
        tmpnn = []
        if len(Nu) > G.number_of_nodes() // 2:
            totalns = (G.number_of_nodes() - len(Nu)) // 2
            #print("Testing neighbors!")
        while cn < totalns:
            nn = random.randint(0, G.number_of_nodes() - 1)
            # non-neighbors of u
            if nn not in Nu and nn not in tmpnn and ((u,nn) not in vlist and (nn,u) not in vlist):
                cn += 1
                x = []
                for d in range(len(X[0])):
                    x.append(X[u][d] - X[nn][d])
                Xdd.append(x)
                Ydd.append(0) # negative sample (edge absent)
                tmpnn.append(nn)
    Xx, Yy = np.array(Xd+Xdd), np.array(Yd+Ydd)
    Xd, Yd = np.array(Xd), np.array(Yd)
    Xdd, Ydd = np.array(Xdd), np.array(Ydd)
    indices = np.array(range(len(Yy)))
    np.random.shuffle(indices)
    Xt = Xx[indices]
    Yt = Yy[indices]
    
    tf = 0.85
    
    CV = int(len(Yt) * tf)
    trainX = Xt[0:CV]
    testX = Xt[CV:]
    trainY = Yt[0:CV]
    testY = Yt[CV:]

    # Train the Random Forest classifier using grid search to find the best hyperparameters
    grid_search = GridSearchCV(RandomForestClassifier(n_estimators=100, min_samples_split=5), {'max_depth': [None, 10, 20, 30]}, cv=5)
    grid_search.fit(trainX, trainY)

    # Test the Random Forest classifier
    modelRF = grid_search.best_estimator_.fit(trainX, trainY)
    predictedY = modelRF.predict(testX)
    predictedProbY = modelRF.predict_proba(Xd)[:, 1]

    # Compute the accuracy score
    acc = accuracy_score(predictedY, testY)
    print("Link Prediction Accuracy:",acc)
   
    return predictedProbY, vlist, acc

In [5]:
# Sparsify (remove edges) the graph for given percentage using the different methods as described below:
# val = 0 ; Random Heuristic = Random deletion of edges
# val = 1 ; Random-Link Prediction Heuristic = Randomly choose edges to delete based on the Link Prediction probabilities 

def sparsify_graph(G, edges, prob, perc, val):
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_to_delete = int(num_edges * perc)
    deleted = [False] * num_edges  # Initialize all edges as not deleted
    
    if val in [0,1]:
        
        if val == 0:    
            prob = [1/num_edges]*num_edges
        
        if val == 1:
            prob = [i/sum(prob) for i in prob]
        
        edges_to_delete = np.random.choice(range(num_edges), size=num_to_delete, replace=False, p=prob)
    
    if val == 2:
        edges_to_delete = list(range(num_to_delete))
        
    for i in edges_to_delete:
        edge = edges[i]
        
        if edge in list(G.edges()):
            G.remove_edge(*edge)
        else:
            edge = (edge[1], edge[0])
            G.remove_edge(*edge)
            
        deleted[i] = True
    
    return G, deleted


In [6]:
# Function to read the node labels of the graph

def read_node_label(filename, skip_head=False):
    fin = open(filename, 'r')
    X = []
    Y = []
    while 1:
        if skip_head:
            fin.readline()
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split(' ')
        X.append(vec[0])
        Y.append(vec[1:])
    fin.close()
    return X, Y

In [7]:
# Fuunction to get best ranker

class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return np.asarray(all_labels)


In [17]:
# Node Classification function

def node_classification(embeddings, G, dataset):
    #X, Y = read_node_label(label,skip_head=True)
    X=list(G.nodes)
    Y=dataset[0].y.tolist()
    Y=[[x] for x in Y]
#     ltrainfrac = [0.05, 0.1, 0.2, 0.3, .4, .5, .6, .7, .8]
    ltrainfrac = [0.8]
    for tf in ltrainfrac:
        print("Training classifier using {:.2f}% nodes...".format(tf * 100))
        acc = split_train_evaluate(X, Y, embeddings, tf)
        
    return acc
        
def split_train_evaluate(X, Y, embeddings, train_precent, seed=0):
    state = np.random.get_state()
    training_size = int(train_precent * len(X))
    np.random.seed(seed)
    shuffle_indices = np.random.permutation(np.arange(len(X)))
    X_train = [X[shuffle_indices[i]] for i in range(training_size)]
    Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]
    X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
    Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

    # train
    binarizer = MultiLabelBinarizer(sparse_output=True)
    binarizer.fit(Y)
    X_tr = [embeddings[x] for x in X_train]
    Y_tr = binarizer.transform(Y_train)
    clf=TopKRanker(LogisticRegression())
    clf.fit(X_tr, Y_tr)

    np.random.set_state(state)
    top_k_list = [len(l) for l in Y_test]
    X_test = np.asarray([embeddings[x] for x in X_test])
    Y_pred = clf.predict(X_test, top_k_list=top_k_list)
    results = {}
    results['acc'] = accuracy_score(binarizer.transform(Y_test),Y_pred)
    print('-------------------')
    print(results)
    print('-------------------')
    
    return results['acc']


In [9]:
# Clustering Evaluation function

def cluster_eval(G, embeddings):
    # converting embedding to a numpy array
    X = [[0] for i in range(G.number_of_nodes())]
    for i in range(0, G.number_of_nodes()):
        X[i] = embeddings[str(i+1)]
    X = np.array(X)

    bestModularity = 0
    bestC = 2
    NOC = 30
    allmodularity = []
    
    for cls in range(2, NOC):
        
        # find clusters using a kmeans clustering algorithm on the embedding
        # Number of clusters is set to cls
        clusters = KMeans(n_clusters=cls, random_state=0).fit(X)
        predG = dict()
        for node in range(len(clusters.labels_)):
            predG[str(node+1)] = clusters.labels_[node]
#         return predG
        # compute the modularity score of the Kmeans clustering
        modularity = comm.community_louvain.modularity(predG, G)
        allmodularity.append(modularity)
#         print("Number of clusters: ", cls, "  Modularity: ", modularity)
        if modularity > bestModularity:
            bestModularity = modularity
            bestC = cls
            
    print("Best Modularity:",bestModularity, "Clusters:", bestC)
    
    return bestModularity


In [10]:
# Get the embeddings of the graph 

def get_embedding(G, walks, embed_size=128, window_size=5, workers=3, iter=5, **kwargs):
    kwargs["sentences"] = walks
    kwargs["min_count"] = kwargs.get("min_count", 0)
    kwargs["vector_size"] = embed_size
    kwargs["sg"] = 1  # skip gram
    kwargs["hs"] = 1  # deepwalk use Hierarchical Softmax
    kwargs["workers"] = workers
    kwargs["window"] = window_size
    kwargs["epochs"] = iter

    print("Learning embedding vectors...")
    model = Word2Vec(**kwargs)
    print("Learning embedding vectors done!")

    embeddings = {}
    for word in G.nodes():
        embeddings[word] = model.wv[word]
    return embeddings

In [11]:
# Function to implement DeepWalk

def deepwalk_walks(G, num_walks, walk_length,):
        nodes = G.nodes()
        walks = []
        for _ in range(num_walks):
            for v in nodes:
                walk = [v]
                while len(walk) < walk_length:
                    cur = walk[-1]
                    cur_nbrs = list(G.neighbors(cur))
                    if len(cur_nbrs) > 0:
                        walk.append(random.choice(cur_nbrs))
                    else:
                        break
                walks.append(walk)
        return walks
    

In [12]:
# Function to implement Node2Vec Walk

def node2vec_walks(G, p, q, num_walks, walk_length,):
        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr].get('weight', 1.0)
                                  for nbr in G.neighbors(node)]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [
                float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = create_alias_table(normalized_probs)

        alias_edges = {}

        for edge in G.edges():
            alias_edges[edge] = get_alias_edge(G, p, q, edge[0], edge[1])
            alias_edges[(edge[1], edge[0])] = get_alias_edge(G, p, q, edge[1], edge[0])
        
        # generate walks
        nodes = G.nodes()
        walks = []
        for _ in range(num_walks):
            for v in nodes:
                walk = [v]
                while len(walk) < walk_length:
                    cur = walk[-1]
                    cur_nbrs = list(G.neighbors(cur))
                    if len(cur_nbrs) > 0:
                        if len(walk) == 1:
#                             print("1.1",alias_nodes[cur][0], alias_nodes[cur][1])
#                             print("1.2",alias_sample(alias_nodes[cur][0], alias_nodes[cur][1]))
                            walk.append(
                                cur_nbrs[alias_sample(alias_nodes[cur][0], alias_nodes[cur][1])])
                        else:
                            prev = walk[-2]
                            edge = (prev, cur)
#                             print("!", edge)
#                             print("2.1",alias_edges[edge][0],
#                                                       alias_edges[edge][1])
#                             print("2.2",alias_sample(alias_edges[edge][0],
#                                                       alias_edges[edge][1]))
                            next_node = cur_nbrs[alias_sample(alias_edges[edge][0],
                                                      alias_edges[edge][1])]
                            walk.append(next_node)
                    else:
                        break
                walks.append(walk)
        return walks

In [13]:
# Helpers for Node2Vec

def create_alias_table(area_ratio):
    """

    :param area_ratio: sum(area_ratio)=1
    :return: accept,alias
    """
    l = len(area_ratio)
    accept, alias = [0] * l, [0] * l
    small, large = [], []
    area_ratio_ = np.array(area_ratio) * l
    for i, prob in enumerate(area_ratio_):
        if prob < 1.0:
            small.append(i)
        else:
            large.append(i)

    while small and large:
        small_idx, large_idx = small.pop(), large.pop()
        accept[small_idx] = area_ratio_[small_idx]
        alias[small_idx] = large_idx
        area_ratio_[large_idx] = area_ratio_[large_idx] - \
            (1 - area_ratio_[small_idx])
        if area_ratio_[large_idx] < 1.0:
            small.append(large_idx)
        else:
            large.append(large_idx)

    while large:
        large_idx = large.pop()
        accept[large_idx] = 1
    while small:
        small_idx = small.pop()
        accept[small_idx] = 1

    return accept, alias

def alias_sample(accept, alias):
    """

    :param accept:
    :param alias:
    :return: sample index
    """
    N = len(accept)
    i = int(np.random.random()*N)
    r = np.random.random()
    if r < accept[i]:
        return i
    else:
        return alias[i]
    
def get_alias_edge(G, p, q, t, v):
        unnormalized_probs = []
        for x in G.neighbors(v):
            weight = G[v][x].get('weight', 1.0)  # w_vx
            if x == t:  # d_tx == 0
                unnormalized_probs.append(weight/p)
            elif G.has_edge(x, t):  # d_tx == 1
                unnormalized_probs.append(weight)
            else:  # d_tx > 1
                unnormalized_probs.append(weight/q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [
            float(u_prob)/norm_const for u_prob in unnormalized_probs]

        return create_alias_table(normalized_probs)